# Clasificador basado en CNN con Keras de TensorFLow
Basado en https://github.com/eisenjulian/nlp_estimator_tutorial/blob/master/nlp_estimators.ipynb

Analizar esto luego: https://www.quora.com/Intuitively-how-does-mini-batch-size-affect-the-performance-of-stochastic-gradient-descent

Esto explica como usa rel pr_curve https://github.com/tensorflow/tensorboard/blob/master/tensorboard/plugins/pr_curve/README.md

In [1]:
import os
import string
import tempfile
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from Unigrams import UnigramaModeloLenguaje
from sklearn.model_selection import train_test_split

from tensorflow.python.keras.preprocessing import sequence
from tensorboard import summary as summary_lib

tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

archivoTweets = 'tweets_limpios.csv'

params = {
            "num_epochs": 100,
            "batch_size": 50,
            "filter_sizes": "3,4,5",
            "embedding_dim": 50,
            "l2_reg_lambda": 0.0,
            "evaluate_every": 100,
            "dropout_keep_prob": 0.6
        }

C:\Users\joax\Anaconda2\envs\py36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


In [23]:
embeddings = {}
with open('fasttext_vocabulario.dat', 'rb') as fr:
    vocab = pickle.load(fr)
embedding = np.load('fasttext_embeddings.npy')
for r, word in enumerate(vocab):
    embeddings[word] = embedding[r]


salg [-0.60252005 -0.13261367 -0.3657695  -0.47437316  0.49584168 -0.16632056
  0.20377141  0.6212192  -0.15386707 -0.21362138  0.42006245  0.4956354
 -0.0378937  -0.31198838 -0.8911585  -0.06294799  0.5411177   0.22641149
  0.0927573  -0.30725864 -0.0941277  -0.33029705 -0.11033237 -0.00395056
  0.09728592  0.5320334  -0.03348267 -0.39325914 -0.8064308   0.579092
 -0.05297757  0.25451463  0.40392807  0.3669902   0.5975481   0.490092
 -0.25228465 -0.15363932  0.04183417  0.53368807 -0.601102   -0.21547675
 -0.28983423 -0.28722063 -0.0019168  -0.49564323  0.15500762 -0.1070075
  0.17036109 -0.26189816  0.955661   -0.6540091  -0.6289086   0.3058876
 -0.33742002 -0.14338087  0.40410146  0.04862902  0.56555456  0.55657506
 -0.06924602  0.5613754   0.52255976  0.11378741 -0.0481295   0.0625466
 -0.9058689  -1.0092931  -0.04528713  0.28780565  0.4420225  -0.33427703
  0.06110063 -0.35997936 -0.19866677  0.14806817  0.22913514  0.21692315
 -0.28373268  0.5418597  -0.3510203  -0.01108535 -0.11

In [2]:
#Implementación propia del método load_data de Keras basado en el del IMDB
#Se le agrega que devuelva el diccionario generado o cargado
#https://github.com/tensorflow/tensorflow/blob/r1.8/tensorflow/python/keras/_impl/keras/datasets/imdb.py
from tensorflow.python.keras._impl.keras.utils.data_utils import get_file
from tensorflow.python.keras._impl.keras.preprocessing.sequence import _remove_long_seq

def load_data(path='tweets_limpios.csv',
              num_words=None,
              skip_top=0,
              maxlen=None,
              seed=113,
              start_char=1,
              oov_char=2,
              index_from=3,
              recalcular=True):
    """Loads the IMDB dataset.
    Arguments:
      path: where to cache the data (relative to `~/.keras/dataset`).
      num_words: max number of words to include. Words are ranked
          by how often they occur (in the training set) and only
          the most frequent words are kept
      skip_top: skip the top N most frequently occurring words
          (which may not be informative).
      maxlen: sequences longer than this will be filtered out.
      seed: random seed for sample shuffling.
      start_char: The start of a sequence will be marked with this character.
          Set to 1 because 0 is usually the padding character.
      oov_char: words that were cut out because of the `num_words`
          or `skip_top` limit will be replaced with this character.
      index_from: index actual words with this index and higher.
      **kwargs: Used for backwards compatibility.
    Returns:
      Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    Raises:
      ValueError: in case `maxlen` is so low
          that no input sequence could be kept.
    Note that the 'out of vocabulary' character is only used for
    words that were present in the training set but are not included
    because they're not making the `num_words` cut here.
    Words that were not seen in the training set but are in the test set
    have simply been skipped.
    """
    #Carga csv
    df = pd.read_csv(path)
    selected = ['Texto', 'Polaridad']
    non_selected = list(set(df.columns) - set(selected))

    df = df.drop(non_selected, axis=1) # Elimina las columnas innecesarias
    df = df.dropna(axis=0, how='any', subset=selected) # Elimina filas con valores null
    df = df.reindex(np.random.permutation(df.index)) # Revuelve el conjunto de datos
    
    #Convierte las etiquetas a un lista consecutiva iniciando en 0
    labels = sorted(list(set(df[selected[1]].tolist())))
    clases = range(0, len(labels))
    label_dict = dict(zip(labels, clases))
    
    #Carga los unigramas del corpus
    if recalcular:
        modeloUnigramas = UnigramaModeloLenguaje(df[selected[0]].tolist())
        diccionario = modeloUnigramas.obtenerDiccionario()
        diccionarioInvertido = modeloUnigramas.obtenerDiccionarioInvertido()
        frecuencias = modeloUnigramas.obtenerVocabularioFrecuencias()
        
        #Exporte lo calculado a un archivo pickle
        diccionarioCompleto = {}
        diccionarioCompleto["Diccionario"] = diccionario
        diccionarioCompleto["DiccionarioInvertido"] = diccionarioInvertido
        diccionarioCompleto["Frecuencias"] = frecuencias
        with open('DiccionariosFrecuencias.pickle', 'wb') as handle:
            pickle.dump(diccionarioCompleto, handle, protocol=pickle.HIGHEST_PROTOCOL)
    else:
        with open('DiccionariosFrecuencias.pickle', 'rb') as handle:
            diccionarioCompleto = pickle.load(handle)
            diccionario = diccionarioCompleto["Diccionario"]
    
    #Reemmplaza las palabras con su indice del diccionario y lo guarda en un nuevo dataset
    xs = []
    for oracion in df[selected[0]].tolist():
        oracion = [diccionario[word] for word in oracion.split()]
        xs.append(oracion)
    
    labels = df[selected[1]].apply(lambda y: label_dict[y]).tolist()

    #Agrega un caracter de inicio de oración y agrega el offset del indice
    if start_char is not None:
        xs = [[start_char] + [w + index_from for w in x] for x in xs]
    elif index_from:
        xs = [[w + index_from for w in x] for x in xs]

    #Ajusta el tamaño de las oraciones al máximo definido
    if maxlen:
        xs, labels = _remove_long_seq(maxlen, xs, labels)
    if not xs:
        raise ValueError('After filtering for sequences shorter than maxlen=' +
                       str(maxlen) + ', no sequence was kept. '
                       'Increase maxlen.')
        
    #Obtiene el tamaño del vocabulario si no hay máximo de palabras definido
    if not num_words:
        num_words = max([max(x) for x in xs])

    #Esta parte quita todas las palabras que esten fuera del tamaño máximo del diccionario
    # by convention, use 2 as OOV word
    # reserve 'index_from' (=3 by default) characters:
    # 0 (padding), 1 (start), 2 (OOV)
    if oov_char is not None:
        xs = [
            [w if (skip_top <= w < num_words) else oov_char for w in x] for x in xs
        ]
    else:
        xs = [[w for w in x if skip_top <= w < num_words] for x in xs]

    return xs, labels, diccionario

In [3]:
vocab_size = 5000
sentence_size = 35
embedding_size = 128
model_dir = tempfile.mkdtemp()

# we assign the first indices in the vocabulary to special tokens that we use
# for padding, as start token, and for indicating unknown words
pad_id = 0
start_id = 1
oov_id = 2
index_offset = 2

print("Loading data...")
#Paso 1: Carga los datos
x, y, diccionario = load_data(
    num_words=vocab_size, start_char=start_id, oov_char=oov_id,
    index_from=index_offset)

#Paso 2: Divide el dataset original en entrenamiento y prueba
x_train_variable, x_test_variable, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

numClases = len(sorted(list(set(y_train))))
    
print(len(y_train), "train sequences")
print(len(y_test), "test sequences")

print("Pad sequences (samples x time)")
x_train = sequence.pad_sequences(x_train_variable, 
                                 maxlen=sentence_size,
                                 truncating='post',
                                 padding='post',
                                 value=pad_id)
x_test = sequence.pad_sequences(x_test_variable, 
                                maxlen=sentence_size,
                                truncating='post',
                                padding='post', 
                                value=pad_id)
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)

Loading data...
6951 train sequences
773 test sequences
Pad sequences (samples x time)
x_train shape: (6951, 35)
x_test shape: (773, 35)


In [5]:
x_len_train = np.array([min(len(x), sentence_size) for x in x_train_variable])
x_len_test = np.array([min(len(x), sentence_size) for x in x_test_variable])

def parser(x, length, y):
    features = {"x": x, "len": length}
    return features, y

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((x_train, x_len_train, y_train))
    dataset = dataset.shuffle(buffer_size=len(x_train_variable))
    dataset = dataset.batch(100)
    dataset = dataset.map(parser)
    dataset = dataset.repeat()
    dataset = dataset.repeat(100)
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((x_test, x_len_test, y_test))
    dataset = dataset.batch(100)
    dataset = dataset.map(parser)
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

In [27]:
all_classifiers = {}
def train_and_evaluate(classifier):
    # Save a reference to the classifier to run predictions later
    all_classifiers[classifier.model_dir] = classifier
    classifier.train(input_fn=train_input_fn, steps=25000)
    eval_results = classifier.evaluate(input_fn=eval_input_fn)
    predictions = np.array([p['class_ids'][0] for p in classifier.predict(input_fn=eval_input_fn)])
        
    # Reset the graph to be able to reuse name scopes
    tf.reset_default_graph() 
    print("Grafo reiniciado")
    # Add a PR summary in addition to the summaries that the classifier writes
    pr = summary_lib.pr_curve('precision_recall', predictions=predictions, labels=y_test, num_thresholds=21)
    with tf.Session() as sess:
        writer = tf.summary.FileWriter(os.path.join(classifier.model_dir, 'eval'), sess.graph)
        writer.add_summary(sess.run(pr), global_step=0)
        writer.close()

In [7]:
#LinearClassifier para referencia
column = tf.feature_column.categorical_column_with_identity('x', vocab_size)
classifier = tf.estimator.LinearClassifier(n_classes=numClases,feature_columns=[column], model_dir=os.path.join(model_dir, 'bow_sparse'))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\joax\\AppData\\Local\\Temp\\tmp5ccyn6cz\\bow_sparse', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001915CD8AC18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
train_and_evaluate(classifier)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\joax\AppData\Local\Temp\tmp_3td90dn\bow_sparse\model.ckpt-25000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 25001 into C:\Users\joax\AppData\Local\Temp\tmp_3td90dn\bow_sparse\model.ckpt.
INFO:tensorflow:loss = 2.940867, step = 25001
INFO:tensorflow:global_step/sec: 96.5967
INFO:tensorflow:loss = 3.333661, step = 25101 (1.036 sec)
INFO:tensorflow:global_step/sec: 136.233
INFO:tensorflow:loss = 3.0935433, step = 25201 (0.737 sec)
INFO:tensorflow:global_step/sec: 124.866
INFO:tensorflow:loss = 4.2880516, step = 25301 (0.798 sec)
INFO:tensorflow:global_step/sec: 133.334
INFO:tensorflow:loss = 5.7931676, step = 25401 (0.750 sec)
INFO:tensorflow:global_step/sec: 116.998
INFO:tensorflow:loss = 2.611353, step = 2550

INFO:tensorflow:loss = 3.6736004, step = 32801 (0.670 sec)
INFO:tensorflow:global_step/sec: 227.364
INFO:tensorflow:loss = 1.5930617, step = 32901 (0.439 sec)
INFO:tensorflow:global_step/sec: 147.235
INFO:tensorflow:loss = 2.2551544, step = 33001 (0.680 sec)
INFO:tensorflow:global_step/sec: 136.791
INFO:tensorflow:loss = 3.2343566, step = 33101 (0.732 sec)
INFO:tensorflow:global_step/sec: 122.726
INFO:tensorflow:loss = 2.0691974, step = 33201 (0.814 sec)
INFO:tensorflow:global_step/sec: 126.6
INFO:tensorflow:loss = 1.4338659, step = 33301 (0.796 sec)
INFO:tensorflow:global_step/sec: 131.412
INFO:tensorflow:loss = 2.4123578, step = 33401 (0.758 sec)
INFO:tensorflow:global_step/sec: 131.584
INFO:tensorflow:loss = 1.4680668, step = 33501 (0.758 sec)
INFO:tensorflow:global_step/sec: 164.373
INFO:tensorflow:loss = 2.691791, step = 33601 (0.607 sec)
INFO:tensorflow:global_step/sec: 192.822
INFO:tensorflow:loss = 1.7248495, step = 33701 (0.520 sec)
INFO:tensorflow:global_step/sec: 126.281
INF

INFO:tensorflow:global_step/sec: 130.217
INFO:tensorflow:loss = 2.664636, step = 41101 (0.771 sec)
INFO:tensorflow:global_step/sec: 134.407
INFO:tensorflow:loss = 3.5591848, step = 41201 (0.741 sec)
INFO:tensorflow:global_step/sec: 130.386
INFO:tensorflow:loss = 2.5435953, step = 41301 (0.772 sec)
INFO:tensorflow:global_step/sec: 118.939
INFO:tensorflow:loss = 1.3354608, step = 41401 (0.839 sec)
INFO:tensorflow:global_step/sec: 125.651
INFO:tensorflow:loss = 1.8029643, step = 41501 (0.879 sec)
INFO:tensorflow:global_step/sec: 114.46
INFO:tensorflow:loss = 1.3329942, step = 41601 (0.788 sec)
INFO:tensorflow:global_step/sec: 179.691
INFO:tensorflow:loss = 2.9302614, step = 41701 (0.574 sec)
INFO:tensorflow:global_step/sec: 120.659
INFO:tensorflow:loss = 1.8544453, step = 41801 (0.814 sec)
INFO:tensorflow:global_step/sec: 132.805
INFO:tensorflow:loss = 2.2843907, step = 41901 (0.749 sec)
INFO:tensorflow:global_step/sec: 128.548
INFO:tensorflow:loss = 1.886659, step = 42001 (0.790 sec)
INF

INFO:tensorflow:loss = 1.35123, step = 49301 (0.789 sec)
INFO:tensorflow:global_step/sec: 130.387
INFO:tensorflow:loss = 1.2290341, step = 49401 (0.789 sec)
INFO:tensorflow:global_step/sec: 129.711
INFO:tensorflow:loss = 1.3862469, step = 49501 (0.762 sec)
INFO:tensorflow:global_step/sec: 143.857
INFO:tensorflow:loss = 2.410121, step = 49601 (0.682 sec)
INFO:tensorflow:global_step/sec: 189.541
INFO:tensorflow:loss = 1.796403, step = 49701 (0.529 sec)
INFO:tensorflow:global_step/sec: 125.648
INFO:tensorflow:loss = 1.1685084, step = 49801 (0.797 sec)
INFO:tensorflow:global_step/sec: 131.069
INFO:tensorflow:loss = 1.2082056, step = 49901 (0.762 sec)
INFO:tensorflow:Saving checkpoints for 50000 into C:\Users\joax\AppData\Local\Temp\tmp_3td90dn\bow_sparse\model.ckpt.
INFO:tensorflow:Loss for final step: 1.2431934.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-04-03:35:45
INFO:tensorflow:Graph was finalized.
INFO:tenso

TypeError:   precision_recall/assert_type/tensor:0 must be of type <dtype: 'bool'>

In [8]:
#DNN genérica de un nivel
word_embedding_column = tf.feature_column.embedding_column(column, dimension=embedding_size)
classifier = tf.estimator.DNNClassifier(n_classes=numClases,
    hidden_units=[100],
    feature_columns=[word_embedding_column], 
    model_dir=os.path.join(model_dir, 'bow_embeddings'))
train_and_evaluate(classifier)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\joax\\AppData\\Local\\Temp\\tmp5ccyn6cz\\bow_embeddings', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001915CD8A5F8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\

INFO:tensorflow:global_step/sec: 259.76
INFO:tensorflow:loss = 0.31822625, step = 7301 (0.385 sec)
INFO:tensorflow:global_step/sec: 258.421
INFO:tensorflow:loss = 0.48497778, step = 7401 (0.388 sec)
INFO:tensorflow:global_step/sec: 241.027
INFO:tensorflow:loss = 0.13159512, step = 7501 (0.415 sec)
INFO:tensorflow:global_step/sec: 252.563
INFO:tensorflow:loss = 0.9124559, step = 7601 (0.395 sec)
INFO:tensorflow:global_step/sec: 244.554
INFO:tensorflow:loss = 0.1790851, step = 7701 (0.409 sec)
INFO:tensorflow:global_step/sec: 260.435
INFO:tensorflow:loss = 0.33701038, step = 7801 (0.384 sec)
INFO:tensorflow:global_step/sec: 260.219
INFO:tensorflow:loss = 1.8441538, step = 7901 (0.384 sec)
INFO:tensorflow:global_step/sec: 255.785
INFO:tensorflow:loss = 0.17167653, step = 8001 (0.392 sec)
INFO:tensorflow:global_step/sec: 270.994
INFO:tensorflow:loss = 0.89031917, step = 8101 (0.368 sec)
INFO:tensorflow:global_step/sec: 270.261
INFO:tensorflow:loss = 0.15935542, step = 8201 (0.371 sec)
INFO

INFO:tensorflow:global_step/sec: 259.09
INFO:tensorflow:loss = 0.093780674, step = 15501 (0.385 sec)
INFO:tensorflow:global_step/sec: 256.439
INFO:tensorflow:loss = 0.05382355, step = 15601 (0.390 sec)
INFO:tensorflow:global_step/sec: 257.096
INFO:tensorflow:loss = 2.0622137, step = 15701 (0.390 sec)
INFO:tensorflow:global_step/sec: 251.296
INFO:tensorflow:loss = 0.050476655, step = 15801 (0.397 sec)
INFO:tensorflow:global_step/sec: 255.858
INFO:tensorflow:loss = 0.05289324, step = 15901 (0.391 sec)
INFO:tensorflow:global_step/sec: 256.463
INFO:tensorflow:loss = 0.057582915, step = 16001 (0.391 sec)
INFO:tensorflow:global_step/sec: 250.668
INFO:tensorflow:loss = 0.07308249, step = 16101 (0.399 sec)
INFO:tensorflow:global_step/sec: 253.185
INFO:tensorflow:loss = 0.24016641, step = 16201 (0.394 sec)
INFO:tensorflow:global_step/sec: 257.772
INFO:tensorflow:loss = 0.050667636, step = 16301 (0.389 sec)
INFO:tensorflow:global_step/sec: 249.422
INFO:tensorflow:loss = 1.0797025, step = 16401 (

INFO:tensorflow:loss = 0.7160475, step = 23601 (0.416 sec)
INFO:tensorflow:global_step/sec: 245.152
INFO:tensorflow:loss = 0.05488596, step = 23701 (0.407 sec)
INFO:tensorflow:global_step/sec: 237.039
INFO:tensorflow:loss = 0.015487539, step = 23801 (0.423 sec)
INFO:tensorflow:global_step/sec: 255.133
INFO:tensorflow:loss = 0.36340305, step = 23901 (0.391 sec)
INFO:tensorflow:global_step/sec: 260.433
INFO:tensorflow:loss = 0.34748206, step = 24001 (0.385 sec)
INFO:tensorflow:global_step/sec: 247.575
INFO:tensorflow:loss = 0.06123223, step = 24101 (0.404 sec)
INFO:tensorflow:global_step/sec: 255.784
INFO:tensorflow:loss = 0.5897621, step = 24201 (0.390 sec)
INFO:tensorflow:global_step/sec: 255.132
INFO:tensorflow:loss = 0.022742748, step = 24301 (0.393 sec)
INFO:tensorflow:global_step/sec: 262.481
INFO:tensorflow:loss = 0.077755526, step = 24401 (0.381 sec)
INFO:tensorflow:global_step/sec: 261.795
INFO:tensorflow:loss = 0.052037932, step = 24501 (0.382 sec)
INFO:tensorflow:global_step/s

TypeError:   precision_recall/assert_type/tensor:0 must be of type <dtype: 'bool'>

In [21]:
head = tf.contrib.estimator.multi_class_head(numClases)

def cnn_model_fn(features, labels, mode, params):    
    input_layer = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=params['embedding_initializer'])
    
    training = mode == tf.estimator.ModeKeys.TRAIN
    dropout_emb = tf.layers.dropout(inputs=input_layer, 
                                    rate=0.2, 
                                    training=training)

    conv = tf.layers.conv1d(
        inputs=dropout_emb,
        filters=32,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    
    # Global Max Pooling
    pool = tf.reduce_max(input_tensor=conv, axis=1)
    
    hidden = tf.layers.dense(inputs=pool, units=250, activation=tf.nn.relu)
    
    dropout_hidden = tf.layers.dropout(inputs=hidden, 
                                       rate=0.2, 
                                       training=training)
    
    logits = tf.layers.dense(inputs=dropout_hidden, units=numClases)
    
    # This will be None when predicting
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])

    optimizer = tf.train.AdamOptimizer()
    
    def _train_op_fn(loss):
        return optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())

    return head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits, 
        train_op_fn=_train_op_fn)
  
params = {'embedding_initializer': tf.random_uniform_initializer(-1.0, 1.0)}
cnn_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                        model_dir=os.path.join(model_dir, 'cnn'),
                                        params=params)

train_and_evaluate(cnn_classifier)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\joax\\AppData\\Local\\Temp\\tmp5ccyn6cz\\cnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000191631BD550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\joax\AppData\Local\Temp\tmp5ccyn6cz\cnn\model.ckpt-1
INFO:tensorflow:Running local_init_op.


TypeError:   precision_recall/assert_type/tensor:0 must be of type <dtype: 'bool'>

In [25]:
def  load_embeddings():
    embeddings = {}
    with open('fasttext_vocabulario.dat', 'rb') as fr:
        vocab = pickle.load(fr)
    embedding = np.load('fasttext_embeddings.npy')
    for r, word in enumerate(vocab):
        embeddings[word] = embedding[r]

    embedding_matrix = np.random.uniform(-1, 1, size=(vocab_size, embedding_size))
    num_loaded = 0
    for w, i in diccionario.items():
        v = embeddings.get(w)
        if v is not None and i < vocab_size:
            embedding_matrix[i] = v
            num_loaded += 1
    print('Successfully loaded pretrained embeddings for '
          f'{num_loaded}/{vocab_size} words.')
    embedding_matrix = embedding_matrix.astype(np.float32)
    return embedding_matrix

embedding_matrix = load_embeddings()

Successfully loaded pretrained embeddings for 5000/5000 words.


In [28]:
def my_initializer(shape=None, dtype=tf.float32, partition_info=None):
    assert dtype is tf.float32
    return embedding_matrix

params = {'embedding_initializer': my_initializer}
cnn_pretrained_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                        model_dir=os.path.join(model_dir, 'cnn_pretrained'),
                                        params=params)
train_and_evaluate(cnn_pretrained_classifier)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\joax\\AppData\\Local\\Temp\\tmp5ccyn6cz\\cnn_pretrained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000019165A94080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\joax\AppData\Local\Temp\tmp5ccyn6cz\cnn_pretrained\model.ckpt-5000
INFO:tensorflo

INFO:tensorflow:global_step/sec: 29.1984
INFO:tensorflow:loss = 0.0002396213, step = 12001 (3.424 sec)
INFO:tensorflow:global_step/sec: 29.3265
INFO:tensorflow:loss = 0.0042429334, step = 12101 (3.412 sec)
INFO:tensorflow:global_step/sec: 28.8955
INFO:tensorflow:loss = 0.06297507, step = 12201 (3.471 sec)
INFO:tensorflow:global_step/sec: 27.8443
INFO:tensorflow:loss = 0.012496441, step = 12301 (3.580 sec)
INFO:tensorflow:global_step/sec: 28.7464
INFO:tensorflow:loss = 0.020481363, step = 12401 (3.479 sec)
INFO:tensorflow:global_step/sec: 28.2046
INFO:tensorflow:loss = 0.00035609928, step = 12501 (3.546 sec)
INFO:tensorflow:global_step/sec: 30.2648
INFO:tensorflow:loss = 0.05104099, step = 12601 (3.303 sec)
INFO:tensorflow:global_step/sec: 29.0294
INFO:tensorflow:loss = 0.010219487, step = 12701 (3.446 sec)
INFO:tensorflow:global_step/sec: 28.3401
INFO:tensorflow:loss = 0.0059721298, step = 12801 (3.530 sec)
INFO:tensorflow:global_step/sec: 28.9456
INFO:tensorflow:loss = 0.0029721754, s

INFO:tensorflow:global_step/sec: 31.5108
INFO:tensorflow:loss = 0.0010893805, step = 20001 (3.174 sec)
INFO:tensorflow:global_step/sec: 31.2944
INFO:tensorflow:loss = 0.05569628, step = 20101 (3.195 sec)
INFO:tensorflow:global_step/sec: 31.7302
INFO:tensorflow:loss = 3.030197e-06, step = 20201 (3.152 sec)
INFO:tensorflow:global_step/sec: 29.9127
INFO:tensorflow:loss = 0.006461341, step = 20301 (3.344 sec)
INFO:tensorflow:global_step/sec: 31.5208
INFO:tensorflow:loss = 1.0340374e-05, step = 20401 (3.173 sec)
INFO:tensorflow:global_step/sec: 31.6501
INFO:tensorflow:loss = 0.010457311, step = 20501 (3.159 sec)
INFO:tensorflow:global_step/sec: 31.6002
INFO:tensorflow:loss = 5.081445e-05, step = 20601 (3.166 sec)
INFO:tensorflow:global_step/sec: 31.1389
INFO:tensorflow:loss = 0.00085106597, step = 20701 (3.211 sec)
INFO:tensorflow:global_step/sec: 31.1874
INFO:tensorflow:loss = 0.008850512, step = 20801 (3.205 sec)
INFO:tensorflow:global_step/sec: 31.0137
INFO:tensorflow:loss = 0.0003218198

INFO:tensorflow:global_step/sec: 31.2944
INFO:tensorflow:loss = 0.0096393265, step = 27901 (3.195 sec)
INFO:tensorflow:global_step/sec: 31.2165
INFO:tensorflow:loss = 0.001808855, step = 28001 (3.203 sec)
INFO:tensorflow:global_step/sec: 31.0618
INFO:tensorflow:loss = 0.023701388, step = 28101 (3.218 sec)
INFO:tensorflow:global_step/sec: 31.5108
INFO:tensorflow:loss = 9.662835e-05, step = 28201 (3.174 sec)
INFO:tensorflow:global_step/sec: 31.6002
INFO:tensorflow:loss = 3.9325373e-06, step = 28301 (3.165 sec)
INFO:tensorflow:global_step/sec: 31.7402
INFO:tensorflow:loss = 0.008410018, step = 28401 (3.152 sec)
INFO:tensorflow:global_step/sec: 31.4219
INFO:tensorflow:loss = 0.0010461499, step = 28501 (3.182 sec)
INFO:tensorflow:global_step/sec: 31.1196
INFO:tensorflow:loss = 0.0024257274, step = 28601 (3.212 sec)
INFO:tensorflow:global_step/sec: 31.5703
INFO:tensorflow:loss = 0.018392883, step = 28701 (3.168 sec)
INFO:tensorflow:global_step/sec: 30.899
INFO:tensorflow:loss = 0.00044702654

TypeError:   precision_recall/assert_type/tensor:0 must be of type <dtype: 'bool'>